In [1]:
imagefolder = '/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/balloon/Data3TypesYminXminYmaxXmax5/images/BF X500K, 05 (3).jpg'
root = '/u/y/u/yuhanl/Downloads/NextGenMaskRCNN-master/code/datasets/balloon/Data3TypesYminXminYmaxXmax5'

In [2]:
import chainer
from chainercv import utils
from chainercv import transforms
import os
import numpy as np
from skimage import exposure, morphology, measure, draw
import json

/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
img = utils.read_image(imagefolder,color=True)
imagename = "BF X500K, 05 (3)"
bbs_file = os.path.join(root, 'bounding_boxes', imagename+'.txt')
bbs = np.stack([line.strip().split() for line in open(bbs_file)]).astype(np.float32)
label = np.stack([0]*bbs.shape[0]).astype(np.int32)



In [4]:

def watershed_image(img,flag):
    """
    use watershed flooding algorithm to extract the loop contour
    :param img: type(numpy.ndarray) image in CHW format
    :return: type(numpy.ndarray) image in HW format
    """
    img_gray = img[1,:,:]
    h, w = img_gray.shape
    img1 = exposure.equalize_hist(img_gray)
    #print(img1.shape)
    # invert the image
    #print("====================")
    #print(np.max(img1).shape)
    img2 = np.max(img1) - img1
    #print("====================")
    #print(img2.shape)
    inner = np.zeros((h, w), np.bool)
    

    centroid = [round(a) for a in findCentroid(img2)]
    inner[centroid[0], centroid[1]] = 1
    min_size = round((h + w) / 20)
    kernel = morphology.disk(min_size)
    inner = morphology.dilation(inner, kernel)

    out = np.zeros((h,w), np.bool)
    out[0, 0] = 1
    out[h - 1, 0] = 1
    out[0, w - 1] = 1
    out[h - 1, w - 1] = 1
    out = morphology.dilation(out, kernel)
    out[0, :] = 1
    out[h - 1, :] = 1
    out[:, w - 1] = 1
    out[:, 0] = 1
    markers = np.zeros((h, w), np.int)
    if flag == 1:
        markers[out] = 1   
    else:
        markers = np.zeros((h, w), np.int)
        markers[inner] = 2
        markers[out] = 1

    labels = morphology.watershed(img2, markers)
    

    return labels

In [5]:
def flood_fitting(img,flag):
    """
    Use watershed flooding algorithm and regional property analysis
    to output the fitted ellipse parameters
    :param img: (numpy.ndarray) image in CHW format
    :return: region property, where property can be accessed through attributes
            example:
            area, bbox, centroid, major_axis_length, minor_axis_length, orientation
    """
    labels = watershed_image(img,flag)
    if flag == 1:
        results = measure.regionprops(labels)
    else:
        results = measure.regionprops(labels - 1)
    
    sorted(results, key=lambda k: k['area'],reverse=True)
    # return the one with largest area
    return results[0]

In [6]:
def cropImage(img, bboxes, expand=True):
    """crop images by the given bounding boxes.

    Args:
        img (numpy.ndarray): image in CHW format
        bboxes (numpy.ndarray): bounding boxes in the format specified by chainerCV
        expand (bool): whether to expand the bounding boxes or not

    Returns:
        a batch of cropped image in CHW format
        The image is in CHW format and its color channel is ordered in
        RGB.

    Return type: list

    """

    if expand:
        _, H, W = img.shape
        bboxes = expand_bbox(bboxes, H, W)

    subimages = list()
    for bbox in bboxes:
        bbox = bbox.astype(np.int)
        subimages.append(img[:, bbox[1]:bbox[3]+1, bbox[2]:bbox[4]+1])
    return subimages, bboxes

def expand_bbox(bbox, H, W):
    """
    expand the bounding box within the range of height and width of the image
    :param bbox: numpy.ndarray bounding box N by 4
    :param H: int Height of the image
    :param W: int Width of the image
    :return: numpy.ndarray expanded bounding box
    """
    b_label = np.zeros(bbox[:, 0].shape)
    b_height = 0.15*(bbox[:, 3] - bbox[:, 1])
    b_width = 0.15*(bbox[:, 4] - bbox[:, 2])
    b_height[b_height < 7] = 7
    b_width[b_width < 7] = 7
    adjust = np.array((b_label, -b_height, -b_width, b_height, b_width)).transpose()
    new_bbox = bbox + adjust
    new_bbox[new_bbox < 0] = 0
    new_bbox[new_bbox[:, 3] >= H, 3] = H - 1
    new_bbox[new_bbox[:, 4] >= W, 4] = W - 1

    return new_bbox

def findCentroid(img):
    """
    find the centroid position of a image by weighted method
    :param img: (numpy.ndarray) image in HW format
    :return: (tuple) (y,x) coordinates of the centroid
    """
    h, w = img.shape
    # TODO: add weighted method later
    return h/2, w/2

In [12]:
def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "balloon":
            return super(self.__class__, self).load_mask(image_id)

            # Convert polygons to a bitmap mask of shape
            # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                            dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
                # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

            # Return mask, and array of class IDs of each instance. Since we have
            # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

In [13]:
bboxes = bbs
All_Image_Defects_List = list()
subimages, bboxes = cropImage(img, bboxes)
current_img_defect_List = list()
defects_Dict = dict()
defects_X_List = list()
defects_Y_List = list()
for subim, bbox in zip(subimages, bboxes):
    region1 = flood_fitting(subim,bbox[0])

    result = (int(region1['centroid'][0]+bbox[1]), int(region1['centroid'][1]+bbox[2]),
                        int(region1['minor_axis_length'] / 2), int(region1['major_axis_length'] / 2),
                        -region1['orientation'])
    rr,cc = draw.ellipse_perimeter(*result)

    assert len(rr) == len(cc)
    for i in range(len(cc)):
        defects_X_List.append(cc[i])
        defects_Y_List.append(rr[i])
    defects_Dict['X'] = defects_X_List
    defects_Dict['Y'] = defects_Y_List
    current_img_defect_List.append(defects_Dict)
All_Image_Defects_List.append(current_img_defect_List)

/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/u/y/u/yuhanl/anaconda3/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


In [15]:
with open("via_region_data.json","w+") as jsonfile:
    each_file = dict()
    for i in range(len(All_Image_Defects_List)):
        img_i = All_Image_Defects_List[i]
        file_size = os.path.getsize(imagefolder)
        filename_size = imagename + str(file_size)
        each_file[filename_size] = {'fileref':"",'size':file_size,'filename':imagename}
        each_file[filename_size].update({'base64_img_data':"",'file_attributes':{},'regions':{}})
        each_bbox = each_file[filename_size]['regions']
        for j in range(len(img_i)):
            each_bbox.update({str(j):{'shape_attributes':{},'region_attributes':{}}})
            each_bbox[str(j)]['shape_attributes'] = {'name':"polygon"}
            each_bbox[str(j)]['shape_attributes'].update({'all_points_x':[],'all_points_y':[]})
            each_bbox[str(j)]['shape_attributes']['all_points_x'] = np.asarray(img_i[j]['X']).tolist()#int64Toint32(img_i[j]['X'])
            each_bbox[str(j)]['shape_attributes']['all_points_y'] = np.asarray(img_i[j]['Y']).tolist()#int64Toint32(img_i[j]['Y'])

    json.dump(each_file,jsonfile)

In [ ]:
config = balloon.BalloonConfig()
BALLOON_DIR = os.path.join(ROOT_DIR, "datasets/balloon")
